# Beer Ratings

In [1]:
# import packages
import pandas as pd
import numpy as np
import sqlite3

# ignore warnings
import warnings
warnings.filterwarnings('ignore') 

# pandas defaults
pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 500)

In [2]:
# create connection
PATH = 'C:\\data\\'
conn = sqlite3.connect(PATH + 'beer.sqlite')

In [3]:
# load data
style = pd.read_sql_query('select * from styles', conn)
brewery = pd.read_sql_query('select * from brewery', conn)
df = pd.read_sql_query('select * from df', conn)

# close connection
conn.close()

In [4]:
# clean data
df['rating'] = pd.to_numeric(df['rating'], downcast='float')
df['brewery_id'] = pd.to_numeric(df['brewery_id'])
df['style'] = df['style'].str.strip()
df.reset_index(inplace = True)

In [5]:
# get valid users
users = df['user_id'].value_counts().rename_axis('user_id').to_frame('counts').reset_index()
# keep only users with 10 ratings or more
users_valid = users[users['counts']>=10]
# keep only top 1000 users based on number of votes
users_top1000 = users.iloc[:1000]

In [6]:
# remove unwanted styles (cider, sake,...)
df_clean = pd.merge(style, df, on='style')

# include brewery location
df_clean = pd.merge(df_clean, brewery, left_on='brewery_id', right_on='id')

In [7]:
# create dataframe with beer info only (without user's review)
beer = df_clean[['division','alias','type','beer_name','beer_id','brewery_name','brewery_id','abv','location']]\
            .drop_duplicates('beer_id')

# create dataframe with ratings aggregated by beer
categories = df_clean.groupby(['beer_id']).agg(appearance = ('appearance', 'mean'), aroma = ('aroma', 'mean'), 
                                                     palate = ('palate', 'mean'), taste = ('taste', 'mean'), 
                                                     overall = ('overall', 'mean')).reset_index()

In [8]:
# remove users with less than 10 votes
rat_valid = pd.merge(users_valid, df_clean, on='user_id')

In [9]:
# standardization of ratings by user
standard = rat_valid.groupby('user_id').transform(lambda x: ((x - x.mean())/x.std())) # time consuming step
standard.reset_index(inplace = True)
standard.rename(columns={'rating':'rating_standard'}, inplace = True)
rat_valid.reset_index(inplace = True)
standard_rat = pd.merge(standard[['level_0', 'rating_standard']], rat_valid[['level_0','rating', 'beer_id']], on='level_0')

In [10]:
# aggregate ratings by beer
standard_rat_agg = standard_rat.groupby(['beer_id']).agg(count = ('rating_standard', 'count'),
                                                        avg_norm = ('rating_standard', 'mean'), 
                                                        std = ('rating', 'std'), avg = ('rating', 'mean')).reset_index()
# remove beers with less than 50 reviews
standard_rat_agg = standard_rat_agg[standard_rat_agg['count']>50]

In [11]:
# create weighted rating
C = 0 # the mean is close to zero due to the standardization
m = 500

standard_rat_agg['wr'] = (standard_rat_agg['count']/(standard_rat_agg['count'] + m))*\
                            standard_rat_agg['avg_norm'] + (m / (standard_rat_agg['count'] + m))*C

In [12]:
# normalize results
standard_rat_agg['score']  = 100*(standard_rat_agg['wr']-standard_rat_agg['wr'].min())/\
                                    (standard_rat_agg['wr'].max()-standard_rat_agg['wr'].min())

standard_rat_agg['popularity']  = 100*(standard_rat_agg['count']-standard_rat_agg['count'].min())/\
                                    (standard_rat_agg['count'].max()-standard_rat_agg['count'].min())

standard_rat_agg['p_dev'] = standard_rat_agg['std']/standard_rat_agg['avg']

In [13]:
# join data
valid_final = pd.merge(standard_rat_agg[['beer_id','count','avg','score','p_dev', 'popularity']], beer, on='beer_id')
valid_final = pd.merge(valid_final, categories, on='beer_id')


# percentile relative to each style (for final score)
type_count = valid_final.groupby(['type']).agg(type_count = ('type', 'count')).reset_index()
type_positions = valid_final.groupby(['type'])[["score"]].rank(method='max').reset_index()
type_positions.rename(columns={"score": "type_rank"}, inplace=True)
valid_final.reset_index(inplace=True)
valid_final = pd.merge(valid_final, type_count, on='type')
valid_final = pd.merge(valid_final, type_positions, on='index')
valid_final['type_score'] = valid_final['type_rank'] / valid_final['type_count'] *100

# overall percentile for score and popularity (for final score)
sz = valid_final['score'].size-1
valid_final['perc'] = valid_final['score'].rank(method='max').apply(lambda x: 100.0*(x-1)/sz)
valid_final['pop_perc'] = valid_final['popularity'].rank(method='max').apply(lambda x: 100.0*(x-1)/sz)

# calculate final scores based on normalized score and percentiles 
valid_final['final_score'] = (valid_final['perc'] + valid_final['score'] + valid_final['type_score'])/3
valid_final['final_popularity'] = (valid_final['popularity'] + valid_final['pop_perc']*2)/3

# create categories
conditions = [valid_final['final_score'] > 90, valid_final['final_score'] > 70,
             valid_final['final_score'] > 50, valid_final['final_score'] > 30]
choices = ['Outstanding', 'Very Good', 'Good', 'Okay']
valid_final['score_category'] = np.select(conditions, choices, default='Poor')

conditions = [valid_final['final_popularity'] > 65, valid_final['final_score'] > 35]
choices = ['Famous', 'Well Known']
valid_final['popularity_category'] = np.select(conditions, choices, default='Niche')

In [14]:
# remove unwanted columns
valid_final.drop(columns=['index', 'popularity', 'score', 'division', 
                            'type_count', 'type_rank', 'perc', 'pop_perc'], inplace=True)

# save data
valid_final.to_csv('beer rating.csv', index=False, encoding='utf-8', decimal=',', sep='\t')

FUTURE IDEAS: 
- correlation between categories (palate, aroma,...) ratings
- reviews by date
- style and abv effect on ratings
- most rated beer by users with few votes